# $$\color{red}{\text{Artificial Intelligence - Camputer Assignment 2}}$$
                                                            Part 1: Genetic
$$\color{lime}{\text{Alireza Javid - 810099011}}$$

## $\color{deepskyblue}{\text{Introduction}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در این تمرین با استفاده از روش هایی که در طبیعت وجود دارند مانند جهش, جفت گیری و ... سعی در حل یک مسئله داریم. همانطور که در درس گفته شد الگوریتم های ژنتیک با ساده سازی هایی سعی در حل یک مسئله نسبتا بزرگ می کنند. در این تمرین ما برای یافتن جواب یک معادله از الگوریتم های ژنتیک مطابق زیر استفاده می کنیم.
</font>
</p>

## $\color{deepskyblue}{\text{Import Libraries}}$

In [2]:
import random
import numpy as np

## $\color{deepskyblue}{\text{Assign Parameters}}$

In [24]:
# TODO assign numbers to variables below
crossoverProbability =  0.5# probabality
carryPercentage =  0.1# prob
populationSize =  200# size

## $\color{deepskyblue}{\text{Modeling}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
<ul dir =rtl>
<ul>
    <li>
    در ابتدا برای شبیه سازی مسئله کروموزم اولیه را به صورت یک رشته از اعداد که میان هر کدام عملگر های 4 گانه است, تعریف می کنیم. </li>
    </li>
    <li>
    سپس جمعیت اولیه را نیز با همین الگو و ساختار به صورت رندم می سازیم.
    </li>
    <li>
    تابع سازگاری را اختلاف اندازه میان مقدار هدف و مقدار هر کروموزوم تعریف می کنیم.
    </li>
    <li>
    کراس اور را به این صورت انجام می دهیم که در ابتدا به صورت رندم مکان شکستن کروموزوم را مشخص می کنیم و سپس دو رشته متفاوت را با هم ادغام می کنیم. باید توجه کنیم که شرط کروموزم بودن اولیه با ادغام کردن نباید شکسته شود. برای میتویشن نیز به صورت رندم یک عدد یا عملگر را تغییر می دهیم
    </li>
    <li>
    الگوریتم ژنتیک پیاده سازی شده در زیر مشخص است.
    </li>
</ul>
</font>
</p>

In [23]:
def get_n_minimums(l, n):
    s = sorted(l)[:n]
    return [l.index(x) for x in s]


class EquationBuilder:

    def __init__(self, operators, operands, equationLength, goalNumber):
        self.operators = operators
        operands = [str(x) for x in operands]
        self.operands = operands
        self.equationLength = equationLength
        self.goalNumber = goalNumber

        # Create the earliest population at the begining
        self.population = self.makeFirstPopulation()

    def makeFirstPopulation(self):
        chromosome_list = []
        for i in range(populationSize):
            operand_list = random.choices(self.operands, k=12)
            operator_list = random.choices(self.operators, k=11)
            chromosome = [item for sublist in zip(operand_list, operator_list) for item in sublist]
            chromosome.append(operand_list[-1])
            chromosome_list.append(chromosome)
        return chromosome_list

    # TODO create random chromosomes to build the early population, and return it

    def findEquation(self):
        # Create a new generation of chromosomes, and make it better in every iteration
        while (True):
            random.shuffle(self.population)

            fitnesses = []
            for i in range(populationSize):
                fitness = self.calcFitness(self.population[i])
                if fitness == 1:
                    return self.population[i]
                fitnesses.append(fitness)

            #calculate the fitness of each chromosome
            self.total_fitness = sum(fitnesses)
            bestChromosome = [self.population[x] for x in
                              get_n_minimums(fitnesses, int(populationSize * carryPercentage))]


            # TODO return chromosome if a solution is found, else save the fitness in an array

            # TODO find the best chromosomes based on their fitnesses, and carry them directly to the next generation (optional)
            carriedChromosomes = []
            for i in range(0, int(populationSize * carryPercentage)):
                carriedChromosomes.append(bestChromosome[i])

            # A pool consisting of potential candidates for mating (crossover and mutation)
            matingPool = self.createMatingPool(populationSize - int(populationSize * carryPercentage), fitnesses)

            # The pool consisting of chromosomes after crossover
            crossoverPool = self.createCrossoverPool(matingPool)

            # Delete the previous population
            self.population.clear()

            # Create the portion of population that is undergone crossover and mutation
            for i in range(populationSize - int(populationSize * carryPercentage)):
                self.population.append(self.mutate(crossoverPool[i]))

            # Add the prominent chromosomes directly to next generation
            self.population.extend(carriedChromosomes)

    def createMatingPool(self, pool_size, fitnesses):
        # TODO make a brand new custom pool to accentuate prominent chromosomes (optional)
        # TODO create the matingPool using custom pool created in the last step and return it
        prob = [x / self.total_fitness for x in fitnesses]
        indexes = list(range(len(fitnesses)))
        selected_index = np.random.choice(indexes, pool_size, p=prob)
        matingPool = [self.population[x] for x in selected_index]
        return matingPool

    def createCrossoverPool(self, matingPool):
        crossoverPool = []
        for i in range(len(matingPool)):
            if random.random() > crossoverProbability:
            # TODO don't perform crossover and add the chromosomes to the next generation directly to crossoverPool
                crossoverPool.append(matingPool[i])
            else:
        # TODO find 2 child chromosomes, crossover, and add the result to crossoverPool
                position = random.randint(1,self.equationLength)
                first_chromosome = random.randint(0, len(matingPool) - 1)
                second_chromosome = random.randint(0, len(matingPool) - 1)
                cross_over = matingPool[first_chromosome][:position] + matingPool[second_chromosome][position:]
                crossoverPool.append(cross_over)
        return crossoverPool

    def mutate(self, chromosome):
        # TODO mutate the input chromosome
        rand = random.randint(0, self.equationLength - 1)
        if chromosome[rand] in self.operands:
            chromosome[rand] = random.choice(self.operands)
        else:
            chromosome[rand] = random.choice(self.operators)
        return chromosome

    def calcFitness(self, chromosome):
        objective = abs(self.goalNumber - eval(''.join(chromosome)))
        return 1 / (1 + objective)


In [25]:
operands = [1,2,3,4,5,6,7,8,9,10]
operators = ['+', '-', '*','%']
equationLength= 23
goalNumber= -123456
equationBuilder = EquationBuilder(operators, operands, equationLength, goalNumber)
equation = equationBuilder.findEquation()
print(''.join(equation), '=', goalNumber)

10-9*10*3*10*9*5-6-5*8*7*7 = -123456



## $\color{deepskyblue}{\text{Answers}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
<ul dir =rtl>
<ol>
  <li>
   اگر جمعیت اولیه بسیار کم باشد زمان ساخت هر نسل و ایتریشن کاهش می‌یابد اما ممکن است بسیار دیر به جواب برسیم چون فضای مسئله بسیار بزرگ است و ما حالات بسیار کمی را پوشش می‌دهیم و در نظر می‌گیریم در نتیجه باید ایتریشن های خیلی زیادی انجام دهیم و نسل های زیادی بسازیم و این کار باعث افزایش چشمگیر زمان اجرای برنامه می‌شود.
اگر جمعیت اولیه بسیار زیاد باشد تعداد ایتریشن ها کمتر می‌شود ولی زمان ساخت هر نسل و هر ایتریشن به مقدار قابل توجهی افزایش می‌یابد و باعث افزایش چشم گیر زمان اجرای برنامه می‌شود، در صورتی که سرعت برنامه دلیل اصلی ما برای استفاده از این الگوریتم و چک نکردن تمام حالات ممکن است. 
  </li><li>
  با این کار دقت برنامه افزایش می‌یابد ولی سرعت آن کاهش پیدا می‌کند.
  </li>
  <li>
  خیر. اگر از crossover استفاده نشود ما به صورت کاملا رندم مکان هر ژن را تغییر می دهیم تا در نهایت به کروموزوم مطلوب برسیم و اجرای برنامه بسیار زمان بر و طولانی خواهد شد. اگر از mutation استفاده نکنیم ممکن است به دلیل جهت گیری در یافتن پاسخ صحیح و در نظر نگرفتن تمامی موارد به پاسخ مطلوب نرسیم و یا دیر تر برسیم. به صورت خلاصه crossover برای افزایش سرعت برنامه و mutation برای افزایش دقت برنامه صورت می گیرد. 
  </li>
  <li>
  جمعیت را به یک حد نصاب میرسانیم و سپس با مقدار قابل قبولی شانس برای جهش ژن ها در نظر می گیریم. مقادیر ابتدایی داده شده با سرعت مناسبی جوابی برای مسئله می دهند.
  </li>
  <li>
   یکی از راه ها افزایش احتمال انجام عمل crossover است و راه دیگر این است که بخشی از جامعه را به صورت تصادفی حذف کنیم و به همین تعداد کروموزوم حذف شده، تعدادی کروموزم جدید، رندوم و بدون بایاس به جمعیت اضافه کنیم. 
  </li>
  <li>
  یک میزان cut off در نظر بگیریم و هنگامی که مقدار fitness ما به این اندازه مطلوبیت رسید (در این مسئله اختلاف دو معادله به حد کمی رسید.) خروجی را بدهیم. یا می توان الگوریتم را بار های محدودی اجرا کرد و در نهایت کروموزوم با بیشترین fitness را خروجی داد.
  </li>
</ol> 
</font>
</p>



## $\color{deepskyblue}{\text{Conclusion}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
برای مسائلی که فضای حالت بسیار بزرگی دارند و چک کردن تمام حالات در آن ها از لحاظ زمانی غیر ممکن است می توان از الگوریتم ژنتیک استفاده کرد که در زمان بسیار کمتر و با دقت بسیار زیاد ما را به پاسخ می‌رساند، دقیقا مانند طبیعت که در آن گونه های برتر که برای بقا از بقیه مناسب تر هستند با گذشت زمان باقی می‌مانند و باقی گونه ها نابود می‌شوند. 
</font>
</p>